# 1. Me conecto con la API de IOL

In [1]:
import requests
from datetime import datetime, timedelta

import pandas as pd

In [2]:
URL__API_IOL__BASE = 'https://api.invertironline.com/'
URL__API_IOL__TOKEN = URL__API_IOL__BASE + 'token'

URL__API_IOL__V2 = URL__API_IOL__BASE + 'api/v2/'
URL__API_IOL__BONDS = URL__API_IOL__V2 + 'Cotizaciones/bonos/bonos/argentina'
URL__API_IOL__CEDEARS = URL__API_IOL__V2 + 'Cotizaciones/acciones/cedears/argentina'
URL__API_IOL__ACCOUNT = URL__API_IOL__V2 + 'estadocuenta'
URL__API_IOL__ASSET_DATA = URL__API_IOL__V2 + '{Mercado}/Titulos/{Simbolo}/Cotizacion'

SECRET__USER_IOL = 'nelsonmaty'
SECRET__PASS_IOL = 'admindora9'

In [3]:
request_token_data = {
    'username': SECRET__USER_IOL,
    'password': SECRET__PASS_IOL,
    'grant_type': 'password'
}

request_token_headers = {
    'Content-Type': 'application/x-www-form-urlencoded'
}

r = requests.post(url=URL__API_IOL__TOKEN, data=request_token_data, headers=request_token_headers)

request_headers = {
    'Authorization': 'Bearer ' + r.json()['access_token']
}

In [4]:
r.json()['access_token']

'PrLc6q_VRHk914s3UsAXVTmMJtkrCwnx8rEQ7RGgCaNXiVLJYq0GITcIULdmjC6j85RMmrfuPaIOWiA6KmvsllYdfggonX3bWp2a3Oe0JkiSRDzcpUUJeSDGGspkzC8qldc0K5Q4ht4J8b4DY7NMYVFfyqoNF3AaIt1gzwW3LqznUIsQDD6wpoG3m-cX5EMrL-fX3rZk2mjGxwhamFPCypK5dW_xg-rF9Ha7lcBLejouMHukrSDN1YIH3wt5uThX23b_OcbAtQLTUXVmYOpIOLq-rYjxs1KIU7UE2Z_IZXLCETx41vs9fmRZsLS3NYfzseP_7PmJwbX22LQFVgy_R2dGwwZY3kcG-r4MLSZ2cFe44Es_38W1v3wydL3kZCBlSKJRzaEfwlq-dTVJCOdg8w'

## 2. Obtengo cotizacion de AL30C en CI

In [25]:
prices = pd.DataFrame(columns=['Date', 'Symbol', 'Bid_T2', 'Offer_T0', 'PnL'])

In [26]:
request_asset_data = {
    'mercado': 'argentina',
    'simbolo': 'AL30C',
    'model.simbolo': 'AL30C',
    'model.mercado': 'bCBA',
    'model.plazo': 't0'
}

puntas_al30C_CI = pd.DataFrame(
    requests.get(
        url=URL__API_IOL__ASSET_DATA,
        params=request_asset_data,
        headers=request_headers
    ).json()['puntas']
)

request_asset_data['model.plazo'] = 't2'

puntas_al30C_T2 = pd.DataFrame(
    requests.get(
        url=URL__API_IOL__ASSET_DATA,
        params=request_asset_data,
        headers=request_headers
    ).json()['puntas']
)

gap = (puntas_al30C_T2.iloc[0]['precioCompra'] - puntas_al30C_CI.iloc[0]['precioVenta']) * 100 / puntas_al30C_CI.iloc[0]['precioVenta']

prices = prices.append({
    'Date': datetime.now(),
    'Symbol': 'AL30C',
    'Bid_T2': puntas_al30C_T2.iloc[0]['precioCompra'],
    'Offer_T0': puntas_al30C_CI.iloc[0]['precioVenta'],
    'PnL': gap
}, ignore_index=True)

<ipython-input-26-dba801cd4d51>:27: RuntimeWarning: divide by zero encountered in double_scalars
  gap = (puntas_al30C_T2.iloc[0]['precioCompra'] - puntas_al30C_CI.iloc[0]['precioVenta']) * 100 / puntas_al30C_CI.iloc[0]['precioVenta']


In [27]:
request_asset_data = {
    'mercado': 'argentina',
    'simbolo': 'AL30D',
    'model.simbolo': 'AL30D',
    'model.mercado': 'bCBA',
    'model.plazo': 't0'
}

puntas_AL30D_CI = pd.DataFrame(
    requests.get(
        url=URL__API_IOL__ASSET_DATA,
        params=request_asset_data,
        headers=request_headers
    ).json()['puntas']
)

request_asset_data['model.plazo'] = 't2'

puntas_AL30D_T2 = pd.DataFrame(
    requests.get(
        url=URL__API_IOL__ASSET_DATA,
        params=request_asset_data,
        headers=request_headers
    ).json()['puntas']
)

gap = (puntas_AL30D_T2.iloc[0]['precioCompra'] - puntas_AL30D_CI.iloc[0]['precioVenta']) * 100 / puntas_AL30D_CI.iloc[0]['precioVenta']

prices = prices.append({
    'Date': datetime.now(),
    'Symbol': 'AL30D',
    'Bid_T2': puntas_AL30D_T2.iloc[0]['precioCompra'],
    'Offer_T0': puntas_AL30D_CI.iloc[0]['precioVenta'],
    'PnL': gap
}, ignore_index=True)

In [28]:
prices

,Date,Symbol,Bid_T2,Offer_T0,PnL
0,2021-07-20 21:48:46.274490,AL30C,36.04,0.0,inf
1,2021-07-20 21:48:48.149607,AL30D,34.70,36.5,-4.931507


In [59]:
df

,simbolo,puntas,ultimoPrecio,variacionPorcentual,apertura,maximo,minimo,ultimoCierre,volumen,cantidadOperaciones,fecha,tipoOpcion,precioEjercicio,fechaVencimiento,mercado,moneda
0,A2E8,None,5301.00,0.00,5301.00,5302.00,5301.00,5301.00,0.0,0.0,2021-07-13T03:00:05.787,None,None,None,BCBA,AR$
1,A2E8D,None,42.50,0.00,42.50,42.50,42.50,42.50,0.0,0.0,2021-07-13T03:00:09.79,None,None,None,BCBA,US$
2,AA21,None,4660.00,0.00,4660.00,0.00,0.00,4660.00,0.0,0.0,2021-07-13T03:00:01.78,None,None,None,BCBA,AR$
3,AA21D,None,45.75,0.00,45.75,0.00,0.00,45.75,0.0,0.0,2021-07-13T03:00:09.177,None,None,None,BCBA,US$
4,AA22,"{'cantidadCompra': 499513.0, 'precioCompra': 9...",99.25,0.05,99.10,99.60,99.00,99.25,260534590.0,54.0,2021-07-13T17:00:10.287,None,None,None,BCBA,AR$
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,TX23,"{'cantidadCompra': 5000000.0, 'precioCompra': ...",154.45,0.22,154.50,154.60,154.15,154.45,191575848.0,59.0,2021-07-13T17:00:30.177,None,None,None,BCBA,AR$
144,TX24,"{'cantidadCompra': 35280.0, 'precioCompra': 14...",145.80,0.13,145.60,146.00,145.00,145.80,500083714.0,74.0,2021-07-13T17:00:16.367,None,None,None,BCBA,AR$
145,TX26,"{'cantidadCompra': 43181.0, 'precioCompra': 11...",118.50,0.67,117.70,118.75,117.25,118.50,776393255.0,157.0,2021-07-13T17:00:11.383,None,None,None,BCBA,AR$
146,TX28,"{'cantidadCompra': 3050.0, 'precioCompra': 111...",112.25,0.22,112.30,112.30,111.50,112.25,23074782.0,28.0,2021-07-13T17:00:11.21,None,None,None,BCBA,AR$


In [60]:
df = df[['simbolo', 'ultimoPrecio', 'moneda']]

In [62]:
df

,simbolo,ultimoPrecio,moneda
0,A2E8,5301.00,AR$
1,A2E8D,42.50,US$
2,AA21,4660.00,AR$
3,AA21D,45.75,US$
4,AA22,99.25,AR$
...,...,...,...
143,TX23,154.45,AR$
144,TX24,145.80,AR$
145,TX26,118.50,AR$
146,TX28,112.25,AR$


In [86]:
dollar_bonds = df[(df['moneda'] == 'US$')][df['simbolo'].str.endswith('D')][df['ultimoPrecio']>0]
peso_bonds = df[(df['moneda'] == 'AR$')][df['ultimoPrecio']>0]

<ipython-input-86-72dc22c0c039>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dollar_bonds = df[(df['moneda'] == 'US$')][df['simbolo'].str.endswith('D')][df['ultimoPrecio']>0]
<ipython-input-86-72dc22c0c039>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dollar_bonds = df[(df['moneda'] == 'US$')][df['simbolo'].str.endswith('D')][df['ultimoPrecio']>0]
<ipython-input-86-72dc22c0c039>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  peso_bonds = df[(df['moneda'] == 'AR$')][df['ultimoPrecio']>0]


In [87]:
dollar_bonds

,simbolo,ultimoPrecio,moneda
1,A2E8D,42.50,US$
3,AA21D,45.75,US$
6,AA25D,32.60,US$
8,AA37D,32.47,US$
10,AA46D,30.25,US$
13,AE38D,36.50,US$
15,AE48D,28.00,US$
17,AF20D,50.00,US$
20,AL29D,36.05,US$
23,AL30D,34.75,US$


In [166]:

peso_prices_list = []

for x in range(len(dollar_bonds)):
    value = peso_bonds[peso_bonds['simbolo'] == dollar_bonds.iloc[x]['simbolo'][:-1]]['ultimoPrecio']
    if (len(value) > 0):
        peso_prices_list.append(value.iloc[0])
    else:
        peso_prices_list.append(float('NaN'))

peso_prices_list


[5301.0,
 4660.0,
 5520.0,
 5100.0,
 3150.0,
 6080.0,
 4850.0,
 8310.0,
 5990.0,
 5747.0,
 5400.0,
 5980.0,
 8650.0,
 7345.0,
 6800.0,
 1280.0,
 11000.0,
 8500.0,
 8350.0,
 6569.0,
 6200.0,
 5590.0,
 6738.5,
 6321.0,
 5495.0,
 4801.0,
 5980.0,
 5100.0,
 nan,
 9225.0,
 126.0]

In [171]:
dollar_bonds['ultimoPrecioPesos'] = peso_prices_list


,simbolo,ultimoPrecio,moneda,ultimoPrecioPesos
1,A2E8D,42.50,US$,5301.0
3,AA21D,45.75,US$,4660.0
6,AA25D,32.60,US$,5520.0
8,AA37D,32.47,US$,5100.0
10,AA46D,30.25,US$,3150.0
13,AE38D,36.50,US$,6080.0
15,AE48D,28.00,US$,4850.0
17,AF20D,50.00,US$,8310.0
20,AL29D,36.05,US$,5990.0
23,AL30D,34.75,US$,5747.0


In [172]:
dollar_bonds.drop(columns='moneda')

,simbolo,ultimoPrecio,ultimoPrecioPesos
1,A2E8D,42.50,5301.0
3,AA21D,45.75,4660.0
6,AA25D,32.60,5520.0
8,AA37D,32.47,5100.0
10,AA46D,30.25,3150.0
13,AE38D,36.50,6080.0
15,AE48D,28.00,4850.0
17,AF20D,50.00,8310.0
20,AL29D,36.05,5990.0
23,AL30D,34.75,5747.0


In [177]:
dollar_bonds['MEP'] = (dollar_bonds['ultimoPrecioPesos'] / dollar_bonds['ultimoPrecio']).round(2)
dollar_bonds.dropna(inplace=True)

dollar_bonds

,simbolo,ultimoPrecio,moneda,ultimoPrecioPesos,MEP
1,A2E8D,42.50,US$,5301.0,124.73
3,AA21D,45.75,US$,4660.0,101.86
6,AA25D,32.60,US$,5520.0,169.33
8,AA37D,32.47,US$,5100.0,157.07
10,AA46D,30.25,US$,3150.0,104.13
13,AE38D,36.50,US$,6080.0,166.58
15,AE48D,28.00,US$,4850.0,173.21
17,AF20D,50.00,US$,8310.0,166.20
20,AL29D,36.05,US$,5990.0,166.16
23,AL30D,34.75,US$,5747.0,165.38


# 3. Resumo

In [107]:
%%time

import requests
import json

import pandas as pd

URL__API_IOL__BASE = 'https://api.invertironline.com/'
URL__API_IOL__TOKEN = URL__API_IOL__BASE + 'token'

URL__API_IOL__V2 = URL__API_IOL__BASE + 'api/v2/'
URL__API_IOL__BONDS = URL__API_IOL__V2 + 'Cotizaciones/bonos/bonos/argentina'
URL__API_IOL__ACCOUNT = URL__API_IOL__V2 + 'estadocuenta'

SECRET__USER_IOL = 'nelsonmaty'
SECRET__PASS_IOL = 'admindora9'

request_token_data = {
    'username': SECRET__USER_IOL,
    'password': SECRET__PASS_IOL,
    'grant_type': 'password'
}

request_token_headers = {
    'Content-Type': 'application/x-www-form-urlencoded'
}

r = requests.post(url=URL__API_IOL__TOKEN, data=request_token_data, headers=request_token_headers)

request_headers = {
    'Authorization': 'Bearer ' + r.json()['access_token']
}

request_bonds_data = {
    'panelCotizacion': {
        'pais': 'argentina',
        'instrumento': 'bonos'
    }
}

r = requests.get(url=URL__API_IOL__BONDS, data=request_token_data, headers=request_headers)
df = pd.DataFrame.from_dict(r.json()['titulos'])

dollar_bonds = df[(df['moneda'] == 'US$')][df['simbolo'].str.endswith('D')][df['ultimoPrecio']>0]
peso_bonds = df[(df['moneda'] == 'AR$')][df['ultimoPrecio']>0]

peso_prices_list = []

for x in range(len(dollar_bonds)):
    value = peso_bonds[peso_bonds['simbolo'] == dollar_bonds.iloc[x]['simbolo'][:-1]]['ultimoPrecio']
    if (len(value) > 0):
        peso_prices_list.append(value.iloc[0])
    else:
        peso_prices_list.append(float('NaN'))

dollar_bonds['ultimoPrecioPesos'] = peso_prices_list
dollar_bonds['MEP'] = (dollar_bonds['ultimoPrecioPesos'] / dollar_bonds['ultimoPrecio']).round(2)

bonds = dollar_bonds[['simbolo', 'MEP']]
bonds.dropna(inplace=True)

print(bonds.sort_values('MEP'))

    simbolo     MEP
46    BPMDD   89.82
3     AA21D  101.86
10    AA46D  104.13
92    PAA0D  107.89
60    DIA0D  113.33
1     A2E8D  124.73
95    PARAD  127.79
139   TVPYD  133.71
98    PAY0D  137.84
8     AA37D  157.07
62    DICAD  163.09
23    AL30D  165.50
13    AE38D  166.13
17    AF20D  166.20
68    GD29D  166.20
71    GD30D  166.31
30    AL41D  166.53
57    CO26D  166.69
80    GD41D  167.05
74    GD35D  167.24
35    AY24D  167.70
20    AL29D  167.72
83    GD46D  167.90
26    AL35D  167.99
77    GD38D  168.59
32    AO20D  168.93
119   T2V1D  169.09
6     AA25D  169.33
15    AE48D  173.21
43    BPLDD  185.79
CPU times: user 138 ms, sys: 55.2 ms, total: 193 ms
Wall time: 2.57 s
<timed exec>:42: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
<timed exec>:42: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
<timed exec>:43: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
<timed exec>:58: SettingWithCop

In [ ]:
46    BPMDD   89.82
3     AA21D  101.86
10    AA46D  104.13
92    PAA0D  107.89
60    DIA0D  113.33
1     A2E8D  124.73
95    PARAD  129.72
139   TVPYD  135.96
98    PAY0D  137.84
8     AA37D  157.07
83    GD46D  161.18
62    DICAD  163.09
68    GD29D  164.40
74    GD35D  166.12
17    AF20D  166.20
20    AL29D  166.25
13    AE38D  166.40
23    AL30D  166.52
57    CO26D  166.58
71    GD30D  166.65
26    AL35D  166.87
119   T2V1D  166.89
77    GD38D  167.00
80    GD41D  167.10
30    AL41D  167.10
32    AO20D  168.46
6     AA25D  169.33
35    AY24D  171.01
15    AE48D  173.21
43    BPLDD  185.79

# 4. Evaluo rendimientos de parking

In [209]:
# Descargo datos historicos desde rava.com

# URL__API_RAVA__HISTORIC_DATA = 'https://clasico.rava.com/lib/restapi/v3/publico/cotizacioneshistoricas/read'

# request_historic_data = {
#     'access-token': 'ab8415b212cb21565d83f159fcbaddba19b64836',
#     'especie': 'AL30',
#     'fecha-inicio': '0000-00-00',
#     'fecha-fin': '2021-07-14'
# }

# data = requests.post(url=URL__API_RAVA__HISTORIC_DATA, data=request_historic_data)

# 5. Utilizo las puntas para calcular valor de venta de mep

## Listado de bonos

In [264]:
request_bonds_data = {
    'panelCotizacion': {
        'pais': 'argentina',
        'instrumento': 'bonos'
    }
}

r = requests.get(url=URL__API_IOL__BONDS, data=request_bonds_data, headers=request_headers)

df = pd.DataFrame.from_dict(r.json()['titulos'])
dollar_bonds = df[(df['moneda'] == 'US$')][df['simbolo'].str.endswith('D')]
dbonds_list = dollar_bonds['simbolo'].values


<ipython-input-264-42127d6d37f6>:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dollar_bonds = df[(df['moneda'] == 'US$')][df['simbolo'].str.endswith('D')]


In [265]:
dbonds_list

array(['A2E8D', 'AA21D', 'AA25D', 'AA37D', 'AA46D', 'AE38D', 'AE48D',
       'AF20D', 'AL29D', 'AL30D', 'AL35D', 'AL41D', 'AO20D', 'AY24D',
       'BPLDD', 'BPMDD', 'CO23D', 'CO26D', 'DIA0D', 'DICAD', 'GD29D',
       'GD30D', 'GD35D', 'GD38D', 'GD41D', 'GD46D', 'PAA0D', 'PARAD',
       'PAY0D', 'PBJ27D', 'PF23D', 'T2V1D', 'TVPYD'], dtype=object)

## MEP AL30

In [32]:
URL__API_IOL__ASSET_DATA = URL__API_IOL__V2 + '{Mercado}/Titulos/{Simbolo}/Cotizacion'

In [33]:
URL__API_IOL__ASSET_DATA

'https://api.invertironline.com/api/v2/{Mercado}/Titulos/{Simbolo}/Cotizacion'

In [98]:
request_asset_data = {
    'mercado': 'argentina',
    'simbolo': 'AL30',
    'model.simbolo': 'AL30',
    'model.mercado': 'bCBA',
    'model.plazo': 't0'
}


In [108]:
al30 = requests.get(
        url=URL__API_IOL__ASSET_DATA,
        params=request_asset_data,
        headers=request_headers
    )

In [109]:
al30.json()

{'ultimoPrecio': 35.37,
 'variacion': 0.7118451025056948,
 'apertura': 35.02,
 'maximo': 35.4,
 'minimo': 35.02,
 'fechaHora': '2021-07-16T17:00:08.9291463-03:00',
 'tendencia': 'sube',
 'cierreAnterior': 35.3882435,
 'montoOperado': 2001251.36,
 'volumenNominal': 5668222,
 'precioPromedio': 0.0,
 'moneda': 'dolar_Estadounidense',
 'precioAjuste': 0.0,
 'interesesAbiertos': 0.0,
 'puntas': [{'cantidadCompra': 3144.0,
   'precioCompra': 34.0,
   'precioVenta': 35.5,
   'cantidadVenta': 650.0},
  {'cantidadCompra': 0.0,
   'precioCompra': 0.0,
   'precioVenta': 35.7,
   'cantidadVenta': 5631.0},
  {'cantidadCompra': 0.0,
   'precioCompra': 0.0,
   'precioVenta': 36.0,
   'cantidadVenta': 21145.0},
  {'cantidadCompra': 0.0,
   'precioCompra': 0.0,
   'precioVenta': 36.5,
   'cantidadVenta': 20760.0},
  {'cantidadCompra': 0.0,
   'precioCompra': 0.0,
   'precioVenta': 36.61,
   'cantidadVenta': 20.0}],
 'cantidadOperaciones': 12}

In [110]:
puntas = pd.DataFrame(al30.json()['puntas'])

In [111]:
puntas_al30 

,cantidadCompra,precioCompra,precioVenta,cantidadVenta
0,172.0,5800.0,5900.0,21700.0
1,293.0,5796.0,5990.0,10461.0
2,280.0,5795.0,5999.0,1000.0
3,345.0,5790.0,6050.0,1715.0
4,346.0,5775.0,6080.0,100.0


In [113]:
request_asset_data['model.simbolo'] = 'AL30D'

puntas_al30D = pd.DataFrame(
    requests.get(
        url=URL__API_IOL__ASSET_DATA,
        params=request_asset_data,
        headers=request_headers
    ).json()['puntas']
)

In [114]:
puntas_al30D

,cantidadCompra,precioCompra,precioVenta,cantidadVenta
0,3144.0,34.0,35.50,650.0
1,0.0,0.0,35.70,5631.0
2,0.0,0.0,36.00,21145.0
3,0.0,0.0,36.50,20760.0
4,0.0,0.0,36.61,20.0


In [115]:
puntas_al30.iloc[0]['precioVenta']

5900.0

In [116]:
compra_mep_al30 = puntas_al30.iloc[0]['precioVenta'] / puntas_al30D.iloc[0]['precioCompra']
venta_mep_al30 = puntas_al30.iloc[0]['precioCompra'] / puntas_al30D.iloc[0]['precioVenta']
compra_mep_al30, venta_mep_al30

(173.52941176470588, 163.38028169014083)

## Itero bonos

In [204]:
mep_dataframe = pd.DataFrame(columns=['Date', 'Symbol', 'Sell_price'])

In [232]:
%%time 
df_dollar_bonds = pd.DataFrame()
df_peso_bonds = pd.DataFrame()
list_of_dollar_bonds = dollar_bonds['simbolo'].values
request_asset_data = {
    'mercado': 'argentina',
    'model.mercado': 'bCBA',
    'model.plazo': 't0'
}
list_of_mep = []

for bond in list_of_dollar_bonds:
    bond_name = bond[:-1]
    print('Requesting', bond_name)
    request_asset_data['simbolo'] = bond_name
    request_asset_data['model.simbolo'] = bond_name
    peso_bond = requests.get(
        url=URL__API_IOL__ASSET_DATA,
        params=request_asset_data,
        headers=request_headers
    )
    if (peso_bond.status_code == 200):
        punta_pesos = pd.DataFrame(peso_bond.json()['puntas'])
        if (len(punta_pesos) != 0):
            print('Requesting', bond)
            request_asset_data['simbolo'] = bond
            request_asset_data['model.simbolo'] = bond
            dollar_bond = requests.get(
                url=URL__API_IOL__ASSET_DATA,
                params=request_asset_data,
                headers=request_headers
            )
            if (dollar_bond.status_code == 200):
                punta_dollar = pd.DataFrame(dollar_bond.json()['puntas'])
                if (
                    len(punta_dollar) != 0 
                ):
                    compra_mep = punta_pesos.iloc[0]['precioVenta'] / punta_dollar.iloc[0]['precioCompra']
                    # venta_mep = punta_pesos.iloc[0]['precioCompra'] / punta_dollar.iloc[0]['precioVenta']
                    mep_dataframe = mep_dataframe.append({
                        'Date': datetime.now(),
                        'Symbol': bond_name,
                        'Buy_price': compra_mep
                        # 'Sell_price': venta_mep
                    }, ignore_index=True)
                else:
                    print('sin punta dollar')
            else:
                print('something went wrong')
        else:
            print('sin punta pesos')
    else:
        print('something went wrong')


Requesting A2E8
sin punta pesos
Requesting AA21
sin punta pesos
Requesting AA25
Requesting AA25D
sin punta dollar
Requesting AA37
Requesting AA37D
sin punta dollar
Requesting AA46
sin punta pesos
Requesting AE38
Requesting AE38D
Requesting AE48
sin punta pesos
Requesting AF20
Requesting AF20D
sin punta dollar
Requesting AL29
Requesting AL29D
Requesting AL30
Requesting AL30D
Requesting AL35
Requesting AL35D
Requesting AL41
Requesting AL41D
Requesting AO20
Requesting AO20D
sin punta dollar
Requesting AY24
Requesting AY24D
Requesting BPLD
sin punta pesos
Requesting BPMD
sin punta pesos
Requesting CO26
Requesting CO26D
Requesting DIA0
sin punta pesos
Requesting DICA
sin punta pesos
Requesting GD29
Requesting GD29D
Requesting GD30
Requesting GD30D
Requesting GD35
Requesting GD35D
Requesting GD38
Requesting GD38D
Requesting GD41
Requesting GD41D
Requesting GD46
Requesting GD46D
Requesting PAA0
sin punta pesos
Requesting PARA
Requesting PARAD
sin punta dollar
Requesting PAY0
sin punta pesos
R

In [233]:
mep_dataframe

,Date,Symbol,Sell_price,Sell_spread_al30,Sell_spread_gd30,Buy_price
0,2021-07-18 02:10:22.405155,AE38,151.272638,-0.155457,-0.179225,NaN
1,2021-07-18 02:10:25.960352,AL29,165.509702,-0.075972,-0.101978,NaN
2,2021-07-18 02:10:27.701042,AL30,164.507042,-0.081570,-0.107418,NaN
3,2021-07-18 02:10:29.441221,AL35,148.630137,-0.170209,-0.193563,NaN
4,2021-07-18 02:10:31.283745,AL41,158.967391,-0.112497,-0.137475,NaN
5,2021-07-18 02:10:33.025541,AO20,inf,inf,inf,NaN
6,2021-07-18 02:10:34.868204,AY24,161.363636,-0.099119,-0.124474,NaN
7,2021-07-18 02:10:38.349671,CO26,0.000000,-1.000000,-1.000000,NaN
8,2021-07-18 02:10:42.650169,GD29,162.345679,-0.093637,-0.119145,NaN
9,2021-07-18 02:10:44.391893,GD30,166.152619,-0.072383,-0.098489,NaN


## Calculo spread contra AL30

In [235]:
r = requests.post(url=URL__API_IOL__TOKEN, data=request_token_data, headers=request_token_headers)

request_headers = {
    'Authorization': 'Bearer ' + r.json()['access_token']
}

In [236]:
r.json()['access_token']

'ft2kgSKLLazGhkR_SSZZ-AtOjEDjznCZjbpdtRzZi45m-M7vdfCSO8a1btAqTZijFkS6gYNlGq1Z5eus383D-9Ob9jJi0yeZBH0YuTlhSU130MhgtAaAkJFkeDbPwVOtANvc42PKfOurcvEM8q3fG6hQsZ7CgtHXPy2c1P3aaQCDur44G9ZPl1RQY3ihZkWXFTPPBCY6l1utDmdiUkb3v5YlAdhyEEQzFwaFUUd5x_r5a6ibLIETE82mk2ys-8R2qj2n6uofi1vyULHdoaoNmblju7HHtsSkWcBBfVkhCkz3kwYbIOvu1as05vXYQuDte3uMInQAbCAdJrj7-F4K3-K_H3kEe70C8D7QjlmOMzxPPWhRFJXmOe_34H9rE8fJmZ3yQFE_eV7ni3BI5YJHMA'

In [198]:
request_asset_data = {
    'mercado': 'argentina',
    'simbolo': 'AL30',
    'model.simbolo': 'AL30',
    'model.mercado': 'bCBA',
    'model.plazo': 't0'
}

puntas_al30 = pd.DataFrame(
    requests.get(
        url=URL__API_IOL__ASSET_DATA,
        params=request_asset_data,
        headers=request_headers
    ).json()['puntas']
)

request_asset_data['model.simbolo'] = 'AL30D'
request_asset_data['simbolo'] = 'AL30D'

puntas_al30D = pd.DataFrame(
    requests.get(
        url=URL__API_IOL__ASSET_DATA,
        params=request_asset_data,
        headers=request_headers
    ).json()['puntas']
)

print(puntas_al30)
print(puntas_al30D)

compra_mep_al30 = puntas_al30.iloc[0]['precioVenta'] / puntas_al30D.iloc[0]['precioCompra']
venta_mep_al30 = puntas_al30.iloc[0]['precioCompra'] / puntas_al30D.iloc[0]['precioVenta']
compra_mep_al30, venta_mep_al30

   cantidadCompra  precioCompra  precioVenta  cantidadVenta
0           200.0        5840.0       6090.0         1000.0
1          6897.0        5772.0       6105.0         1000.0
2          1372.0        5764.0       6200.0         1000.0
3           180.0        5700.0       6329.0         2875.0
4           800.0        5610.0       5894.0         6000.0
   cantidadCompra  precioCompra  precioVenta  cantidadVenta
0          3144.0          34.0        35.50          650.0
1             0.0           0.0        35.70         5631.0
2             0.0           0.0        36.00        21145.0
3             0.0           0.0        36.50        20760.0
4             0.0           0.0        36.61           20.0


(179.11764705882354, 164.50704225352112)

In [208]:
mep_dataframe['Sell_spread_al30'] = (mep_dataframe['Sell_price'] / compra_mep_al30) - 1

In [209]:
mep_dataframe

,Date,Symbol,Sell_price,Sell_spread_al30
0,2021-07-18 02:10:22.405155,AE38,151.272638,-0.155457
1,2021-07-18 02:10:25.960352,AL29,165.509702,-0.075972
2,2021-07-18 02:10:27.701042,AL30,164.507042,-0.081570
3,2021-07-18 02:10:29.441221,AL35,148.630137,-0.170209
4,2021-07-18 02:10:31.283745,AL41,158.967391,-0.112497
5,2021-07-18 02:10:33.025541,AO20,inf,inf
6,2021-07-18 02:10:34.868204,AY24,161.363636,-0.099119
7,2021-07-18 02:10:38.349671,CO26,0.000000,-1.000000
8,2021-07-18 02:10:42.650169,GD29,162.345679,-0.093637
9,2021-07-18 02:10:44.391893,GD30,166.152619,-0.072383


## Agrego GD30



In [210]:
request_asset_data = {
    'mercado': 'argentina',
    'simbolo': 'GD30',
    'model.simbolo': 'GD30',
    'model.mercado': 'bCBA',
    'model.plazo': 't0'
}

puntas_gd30 = pd.DataFrame(
    requests.get(
        url=URL__API_IOL__ASSET_DATA,
        params=request_asset_data,
        headers=request_headers
    ).json()['puntas']
)

request_asset_data['model.simbolo'] = 'GD30D'
request_asset_data['simbolo'] = 'GD30D'

puntas_gd30D = pd.DataFrame(
    requests.get(
        url=URL__API_IOL__ASSET_DATA,
        params=request_asset_data,
        headers=request_headers
    ).json()['puntas']
)

print(puntas_gd30)
print(puntas_gd30D)

compra_mep_gd30 = puntas_gd30.iloc[0]['precioVenta'] / puntas_gd30D.iloc[0]['precioCompra']
venta_mep_gd30 = puntas_gd30.iloc[0]['precioCompra'] / puntas_gd30D.iloc[0]['precioVenta']
compra_mep_gd30, venta_mep_gd30

   cantidadCompra  precioCompra  precioVenta  cantidadVenta
0          2700.0        6249.0       6799.0          156.0
1         11453.0        6251.0       6286.5        15907.0
2         16978.0        6258.5       6287.0        15906.0
3         15986.0        6255.5       6288.0        15903.0
4         15988.0        6254.5       6291.5        15894.0
   cantidadCompra  precioCompra  precioVenta  cantidadVenta
0        108000.0         36.89        37.61        41686.0
1         50000.0         37.21        37.62       108127.0
2            37.0         36.90        37.65         4458.0
3         75000.0         36.95        37.66        53371.0
4          8000.0         36.50        37.67         3190.0


(184.3046896177826, 166.15261898431268)

In [211]:
mep_dataframe['Sell_spread_gd30'] = (mep_dataframe['Sell_price'] / compra_mep_gd30) - 1

In [212]:
mep_dataframe

,Date,Symbol,Sell_price,Sell_spread_al30,Sell_spread_gd30
0,2021-07-18 02:10:22.405155,AE38,151.272638,-0.155457,-0.179225
1,2021-07-18 02:10:25.960352,AL29,165.509702,-0.075972,-0.101978
2,2021-07-18 02:10:27.701042,AL30,164.507042,-0.081570,-0.107418
3,2021-07-18 02:10:29.441221,AL35,148.630137,-0.170209,-0.193563
4,2021-07-18 02:10:31.283745,AL41,158.967391,-0.112497,-0.137475
5,2021-07-18 02:10:33.025541,AO20,inf,inf,inf
6,2021-07-18 02:10:34.868204,AY24,161.363636,-0.099119,-0.124474
7,2021-07-18 02:10:38.349671,CO26,0.000000,-1.000000,-1.000000
8,2021-07-18 02:10:42.650169,GD29,162.345679,-0.093637,-0.119145
9,2021-07-18 02:10:44.391893,GD30,166.152619,-0.072383,-0.098489


In [219]:
mep_dataframe.reset_index(drop=True, inplace=True)

In [226]:
datetime.today().strftime('%Y-%m-%d')

'2021-07-19'

1626665470.451236

In [220]:
mep_dataframe

,Date,Symbol,Sell_price,Sell_spread_al30,Sell_spread_gd30
0,2021-07-18 02:10:22.405155,AE38,151.272638,-0.155457,-0.179225
1,2021-07-18 02:10:25.960352,AL29,165.509702,-0.075972,-0.101978
2,2021-07-18 02:10:27.701042,AL30,164.507042,-0.081570,-0.107418
3,2021-07-18 02:10:29.441221,AL35,148.630137,-0.170209,-0.193563
4,2021-07-18 02:10:31.283745,AL41,158.967391,-0.112497,-0.137475
5,2021-07-18 02:10:33.025541,AO20,inf,inf,inf
6,2021-07-18 02:10:34.868204,AY24,161.363636,-0.099119,-0.124474
7,2021-07-18 02:10:38.349671,CO26,0.000000,-1.000000,-1.000000
8,2021-07-18 02:10:42.650169,GD29,162.345679,-0.093637,-0.119145
9,2021-07-18 02:10:44.391893,GD30,166.152619,-0.072383,-0.098489


## Pruebo con CEDEARs

In [285]:
## LISTADO DE CEDEARS
request_cedears_data = {
    'panelCotizacion': {
        'pais': 'argentina',
        'instrumento': 'acciones',
        'panel': 'cedears'
    }
}

r = requests.get(url=URL__API_IOL__CEDEARS, data=request_cedears_data, headers=request_headers)

df = pd.DataFrame.from_dict(r.json()['titulos'])
dollar_cedears = df[(df['moneda'] == 'US$')][df['simbolo'].str.endswith('D')]
dcedears_list = dollar_cedears['simbolo'].values

<ipython-input-285-a4f2dd4c3ce2>:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dollar_cedears = df[(df['moneda'] == 'US$')][df['simbolo'].str.endswith('D')]


In [288]:
dcedears_list

array(['AAPLD', 'ABBVD', 'ABTD', 'AMDD', 'AMXD', 'AMZND', 'AUYD', 'AVGOD',
       'AXPD', 'AZND', 'BA.CD', 'BABAD', 'BAD', 'BBDD', 'BIDUD', 'BIOXD',
       'BPD', 'BRKBD', 'C.D', 'CAHD', 'CATD', 'CSCOD', 'CVXD', 'DESPD',
       'DISND', 'DOCUD', 'EBAYD', 'EFXD', 'ERJD', 'ETSYD', 'FBD', 'FDXD',
       'FMXD', 'GILDD', 'GLNTD', 'GOGLD', 'GOLDD', 'GSD', 'GSKD', 'HMYD',
       'HWMD', 'IBMD', 'INTCD', 'ITUBD', 'JNJD', 'JPMD', 'KOD', 'LMTD',
       'MAD', 'MCDD', 'MELID', 'MMMD', 'MOD', 'MRKD', 'MSFTD', 'NEMD',
       'NFLXD', 'NKED', 'NVDAD', 'NVSD', 'OGZDD', 'PBRD', 'PFED', 'PGD',
       'PSXD', 'PYPLD', 'QCOMD', 'RIOD', 'SHOPD', 'SLBD', 'SNOWD',
       'SPOTD', 'SQD', 'TD', 'TEND', 'TMD', 'TSLAD', 'TWTRD', 'TXRD',
       'UNHD', 'UNPD', 'VALED', 'VD', 'VISTD', 'VZD', 'WFCD', 'WMTD',
       'XD', 'XOMD', 'ZMD'], dtype=object)

In [289]:
r = requests.post(url=URL__API_IOL__TOKEN, data=request_token_data, headers=request_token_headers)

request_headers = {
    'Authorization': 'Bearer ' + r.json()['access_token']
}

In [297]:
%%time 
df_dollar_cedears = pd.DataFrame()
df_peso_cedears = pd.DataFrame()
list_of_dollar_cedears = dollar_cedears['simbolo'].values
request_asset_data = {
    'mercado': 'argentina',
    'model.mercado': 'bCBA',
    'model.plazo': 't0'
}
mep_dataframe_cedears = pd.DataFrame()

list_of_mep = []

for cedear in list_of_dollar_cedears:
    cedear_name = cedear[:-1]
    print('Requesting', cedear_name)
    request_asset_data['simbolo'] = cedear_name
    request_asset_data['model.simbolo'] = cedear_name
    peso_cedear = requests.get(
        url=URL__API_IOL__ASSET_DATA,
        params=request_asset_data,
        headers=request_headers
    )
    if (peso_cedear.status_code == 200):
        punta_pesos = pd.DataFrame(peso_cedear.json()['puntas'])
        if (len(punta_pesos) != 0):
            print('Requesting', cedear)
            request_asset_data['simbolo'] = cedear
            request_asset_data['model.simbolo'] = cedear
            dollar_cedear = requests.get(
                url=URL__API_IOL__ASSET_DATA,
                params=request_asset_data,
                headers=request_headers
            )
            if (dollar_cedear.status_code == 200):
                punta_dollar = pd.DataFrame(dollar_cedear.json()['puntas'])
                if (
                    len(punta_dollar) != 0 
                ):
                    # compra_mep = punta_pesos.iloc[0]['precioVenta'] / punta_dollar.iloc[0]['precioCompra']
                    venta_mep = punta_pesos.iloc[0]['precioCompra'] / punta_dollar.iloc[0]['precioVenta']
                    mep_dataframe_cedears = mep_dataframe_cedears.append({
                        'Date': datetime.now(),
                        'Symbol': cedear_name,
                        # 'Buy_price': compra_mep
                        'Sell_price': venta_mep
                    }, ignore_index=True)
                else:
                    print('sin punta dollar')
            else:
                print('something went wrong')
        else:
            print('sin punta pesos')
    else:
        print('something went wrong')

mep_dataframe_cedears.to_excel('cedears.xlsx')

Requesting AAPL
Requesting AAPLD
Requesting ABBV
Requesting ABBVD
<timed exec>:40: RuntimeWarning: divide by zero encountered in double_scalars
Requesting ABT
Requesting ABTD
sin punta dollar
Requesting AMD
Requesting AMDD
Requesting AMX
Requesting AMXD
sin punta dollar
Requesting AMZN
Requesting AMZND
Requesting AUY
Requesting AUYD
Requesting AVGO
Requesting AVGOD
sin punta dollar
Requesting AXP
Requesting AXPD
Requesting AZN
Requesting AZND
Requesting BA.C
Requesting BA.CD
Requesting BABA
Requesting BABAD
Requesting BA
Requesting BAD
sin punta dollar
Requesting BBD
Requesting BBDD
Requesting BIDU
Requesting BIDUD
sin punta dollar
Requesting BIOX
Requesting BIOXD
Requesting BP
Requesting BPD
sin punta dollar
Requesting BRKB
Requesting BRKBD
<timed exec>:40: RuntimeWarning: invalid value encountered in double_scalars
Requesting C.
something went wrong
Requesting CAH
Requesting CAHD
sin punta dollar
Requesting CAT
Requesting CATD
Requesting CSCO
Requesting CSCOD
Requesting CVX
Requestin

In [298]:
mep_dataframe_cedears.to_excel('cedears.xlsx')

In [273]:
print(r.json()['access_token'])


lyqXceXnurQqDoIIbjbi3cu9Otj2rW94HWJfcBU3lUi9A4Sn9pDhIC9oq-E2Q-EmVDQYpF_8J5NgT7YvmpeaUOTxpMDdy34XNWDrp2ZLZU_CrHrSc9_sH9BiOF9FbMsmo9u6faZHK1h63_c9CjqDi0-fnENVwoZlzkQ2JWHVtJPctVw01Gum5QdO-DQajyiUJheBvElqow8fZ1aJWL2Fm1UqPc_vEpqtuFxGPoenc2D-5vXQN1lu2ISmsmL_iP8s-SM9HugwMxEZB1ZaMmaHqAgcSjKcrbGFljj3S2eD3R0BRUvD8rHg6UKIFugZzC8QaIJyor7ZRbl2HpC6UiNFVV-Ejlunwp8gDpOFXqjJQ2UAHYA8jaYum7_wBEu1QuC-jspbls0jY5qwgtS8okgQ1w


## Reviso gaps entre CI y T+2

In [8]:
from time import sleep

r = requests.post(url=URL__API_IOL__TOKEN, data=request_token_data, headers=request_token_headers)

request_headers = {
    'Authorization': 'Bearer ' + r.json()['access_token']
}
count = 0
gaps2 = []

while(count != 25):
    count += 1
    request_asset_data = {
        'mercado': 'argentina',
        'simbolo': 'AL30C',
        'model.simbolo': 'AL30C',
        'model.mercado': 'bCBA',
        'model.plazo': 't0'
    }

    puntas_al30C_CI = pd.DataFrame(
        requests.get(
            url=URL__API_IOL__ASSET_DATA,
            params=request_asset_data,
            headers=request_headers
        ).json()['puntas']
    )

    request_asset_data['model.plazo'] = 't2'

    puntas_al30C_T2 = pd.DataFrame(
        requests.get(
            url=URL__API_IOL__ASSET_DATA,
            params=request_asset_data,
            headers=request_headers
        ).json()['puntas']
    )
    gap = (puntas_al30C_T2.iloc[0]['precioCompra'] - puntas_al30C_CI.iloc[0]['precioVenta']) * 100 / puntas_al30C_CI.iloc[0]['precioVenta']
    gaps2.append(gap)
    print('Compra', puntas_al30C_CI.iloc[0]['precioVenta'])
    print('Venta', puntas_al30C_T2.iloc[0]['precioCompra'])
    print(count, 'Gap:', gap, '%')
    sleep(30)




NameError: name 'URL__API_IOL__ASSET_DATA' is not defined

# Como ejecuto una operacion de compra/venta en IOL?

In [31]:
URL__API_IOL__BUY = URL__API_IOL__V2 + 'operar/Comprar'
URL__API_IOL__SELL = URL__API_IOL__V2 + 'operar/Vender'

COMISSION = (0.31 * 2 * 1.21) / 100

r = requests.post(url=URL__API_IOL__TOKEN, data=request_token_data, headers=request_token_headers)

request_headers = {
    'Authorization': 'Bearer ' + r.json()['access_token']
}
request_asset_data = {
    'mercado': 'argentina',
    'model.mercado': 'bCBA',
}
request_asset_data['simbolo'] = 'AL30D'
request_asset_data['model.simbolo'] = 'AL30D'
request_asset_data['model.plazo'] = 't0'

puntas_CI = pd.DataFrame(
    requests.get(
        url=URL__API_IOL__ASSET_DATA,
        params=request_asset_data,
        headers=request_headers
    ).json()['puntas']
)

available_ammount_buy = puntas_CI.iloc[0]['precioVenta'] * puntas_CI.iloc[0]['cantidadVenta']

request_asset_data['simbolo'] = 'AL30C'
request_asset_data['model.simbolo'] = 'AL30C'
request_asset_data['model.plazo'] = 't2'


puntas_T2 = pd.DataFrame(
    requests.get(
        url=URL__API_IOL__ASSET_DATA,
        params=request_asset_data,
        headers=request_headers
    ).json()['puntas']
)

available_ammount_sell = puntas_T2.iloc[0]['precioCompra'] * puntas_T2.iloc[0]['cantidadCompra']
available_ammount_capital = 1000.0 * (1-COMISSION)
ammount = min([available_ammount_buy, available_ammount_sell, available_ammount_capital])
quantity = min([puntas_T2.iloc[0]['cantidadCompra'], puntas_CI.iloc[0]['cantidadVenta'], int(available_ammount_capital / puntas_CI.iloc[0]['precioVenta'])])

valide = datetime.now() + timedelta(days=1)
valide_str = datetime.strftime(valide, '%Y-%m-%d')


request_buy_data = {
  'mercado': 'bCBA',
  'simbolo': 'AL30D',
  'precio': puntas_CI.iloc[0]['precioVenta'],
  'plazo': 't2',
  'tipoOrden': 'precioLimite',
  'cantidad': quantity,
  'validez': valide_str
}


r = requests.post(url=URL__API_IOL__BUY, data=request_buy_data, headers=request_headers)

r.json()

{'ok': False,
 'messages': [{'title': '',
   'description': 'No está disponible la venta de títulos valores con liquidación en Dólares via APIs. Se requiere Declaración Jurada'}]}

27


# Como conozco el estado de la ultima operacion?

# Como veo el saldo de mi cuenta?

# Como puedo operar con websocket?

# Cuan rentable es hacer rulo cable?

In [18]:
request_asset_data = {
    'mercado': 'argentina',
    'simbolo': 'AL30C',
    'model.simbolo': 'AL30C',
    'model.mercado': 'bCBA',
    'model.plazo': 't0'
}

puntas_al30C_CI = pd.DataFrame(
    requests.get(
        url=URL__API_IOL__ASSET_DATA,
        params=request_asset_data,
        headers=request_headers
    ).json()['puntas']
)

request_asset_data['simbolo'] = 'AL30D'
request_asset_data['model.simbolo'] = 'AL30D'

puntas_al30D_CI = pd.DataFrame(
    requests.get(
        url=URL__API_IOL__ASSET_DATA,
        params=request_asset_data,
        headers=request_headers
    ).json()['puntas']
)

# gap = (puntas_al30C_T2.iloc[0]['precioCompra'] - puntas_al30C_CI.iloc[0]['precioVenta']) * 100 / puntas_al30C_CI.iloc[0]['precioVenta']

# prices = prices.append({
#     'Date': datetime.now(),
#     'Symbol': 'AL30C',
#     'Bid_T2': puntas_al30C_T2.iloc[0]['precioCompra'],
#     'Offer_T0': puntas_al30C_CI.iloc[0]['precioVenta'],
#     'PnL': gap
# }, ignore_index=True)

request_asset_data['simbolo'] = 'GD30D'
request_asset_data['model.simbolo'] = 'GD30D'

puntas_GD30D_CI = pd.DataFrame(
    requests.get(
        url=URL__API_IOL__ASSET_DATA,
        params=request_asset_data,
        headers=request_headers
    ).json()['puntas']
)

request_asset_data['simbolo'] = 'GD30C'
request_asset_data['model.simbolo'] = 'GD30C'

puntas_GD30C_CI = pd.DataFrame(
    requests.get(
        url=URL__API_IOL__ASSET_DATA,
        params=request_asset_data,
        headers=request_headers
    ).json()['puntas']
)

In [19]:
puntas_al30D_CI.iloc[0]

cantidadCompra     1780.00
precioCompra         35.85
precioVenta          35.88
cantidadVenta     28336.00
Name: 0, dtype: float64

In [20]:
puntas_al30C_CI.iloc[0]

cantidadCompra    31527.00
precioCompra         35.16
precioVenta          35.21
cantidadVenta       269.00
Name: 0, dtype: float64

In [21]:
puntas_GD30D_CI.iloc[0]

cantidadCompra     6805.00
precioCompra         38.52
precioVenta          38.54
cantidadVenta     14046.00
Name: 0, dtype: float64

In [23]:
puntas_GD30C_CI.iloc[0]

cantidadCompra    22792.00
precioCompra         37.00
precioVenta          37.12
cantidadVenta     44000.00
Name: 0, dtype: float64

In [26]:
(puntas_al30C_CI.iloc[0]['precioCompra'] - puntas_al30D_CI.iloc[0]['precioVenta']) 

-0.720000000000006

In [27]:
(puntas_GD30D_CI.iloc[0]['precioCompra'] - puntas_GD30C_CI.iloc[0]['precioVenta']) 

1.4000000000000057

In [29]:
((puntas_GD30D_CI.iloc[0]['precioCompra'] - puntas_GD30C_CI.iloc[0]['precioVenta']) + (puntas_al30C_CI.iloc[0]['precioCompra'] - puntas_al30D_CI.iloc[0]['precioVenta']))/puntas_al30D_CI.iloc[0]['precioVenta']

0.01895206243032329

In [ ]:
## Pago -0.72 centavos de dolar para ganar 1.40. Retorno de 1,8% (sin comisiones)

In [35]:
from time import sleep

r = requests.post(url=URL__API_IOL__TOKEN, data=request_token_data, headers=request_token_headers)

request_headers = {
    'Authorization': 'Bearer ' + r.json()['access_token']
}
count = 0
pnl_list = []

while(count != 27):
    count += 1
    request_asset_data = {
        'mercado': 'argentina',
        'simbolo': 'AL30C',
        'model.simbolo': 'AL30C',
        'model.mercado': 'bCBA',
        'model.plazo': 't0'
    }

    puntas_al30C_CI = pd.DataFrame(
        requests.get(
            url=URL__API_IOL__ASSET_DATA,
            params=request_asset_data,
            headers=request_headers
        ).json()['puntas']
    )

    request_asset_data['simbolo'] = 'AL30D'
    request_asset_data['model.simbolo'] = 'AL30D'

    puntas_al30D_CI = pd.DataFrame(
        requests.get(
            url=URL__API_IOL__ASSET_DATA,
            params=request_asset_data,
            headers=request_headers
        ).json()['puntas']
    )

    request_asset_data['simbolo'] = 'GD30D'
    request_asset_data['model.simbolo'] = 'GD30D'

    puntas_GD30D_CI = pd.DataFrame(
        requests.get(
            url=URL__API_IOL__ASSET_DATA,
            params=request_asset_data,
            headers=request_headers
        ).json()['puntas']
    )

    request_asset_data['simbolo'] = 'GD30C'
    request_asset_data['model.simbolo'] = 'GD30C'

    puntas_GD30C_CI = pd.DataFrame(
        requests.get(
            url=URL__API_IOL__ASSET_DATA,
            params=request_asset_data,
            headers=request_headers
        ).json()['puntas']
    )
    pnl = ((puntas_GD30D_CI.iloc[0]['precioCompra'] - puntas_GD30C_CI.iloc[0]['precioVenta']) + (puntas_al30C_CI.iloc[0]['precioCompra'] - puntas_al30D_CI.iloc[0]['precioVenta']))*100/puntas_al30D_CI.iloc[0]['precioVenta']
    pnl_list.append(pnl)
    print(count, 'PNL:', pnl, '%')
    sleep(30)




1 PNL: 2.13651498335184 %


KeyboardInterrupt: 

In [ ]:
# 1 PNL: 2.4233983286908205 %
# 2 PNL: 2.479799387015883 %
# 3 PNL: 2.479799387015883 %
# 4 PNL: 2.3955431754874636 %
# 5 PNL: 2.4233983286908205 %
# 6 PNL: 2.480490523968786 %
# 7 PNL: 2.3962106436333226 %
# 8 PNL: 2.3962106436333226 %
# 9 PNL: 2.3962106436333226 %
# 10 PNL: 2.4240735580941695 %
# 11 PNL: 2.452619843924179 %
# 12 PNL: 2.4247491638795915 %
# 13 PNL: 2.4247491638795915 %
# 14 PNL: 2.3962106436333226 %
# 15 PNL: 2.4247491638795915 %
# 16 PNL: 2.4247491638795915 %
# 17 PNL: 2.3962106436333226 %
# 18 PNL: 2.3955431754874836 %
# 19 PNL: 2.5076623014767305 %
# 20 PNL: 2.4797993870158637 %
# 21 PNL: 2.4791086350974947 %
# 22 PNL: 2.4791086350974947 %
# 23 PNL: 2.4791086350974947 %
# 24 PNL: 2.5076623014767305 %
# 25 PNL: 2.591973244147156 %